In [9]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)


In [10]:
input_query=inputs[1]
input_query

tensor([0.5500, 0.8700, 0.6600])

In [11]:
input_1=inputs[0]

In [12]:
torch.dot(input_query, input_1)

tensor(0.9544)

In [13]:
i=0 

res=torch.dot(inputs[i], input_query)
res

tensor(0.9544)

In [14]:
query = inputs[1]  # 2nd input token is the query

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # dot product (transpose not necessary here since they are 1-dim vectors)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [ ]:
attn_weights_2_tmp=attn_scores_2/attn_scores_2.sum()


In [17]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

softmax_naive(attn_scores_2)

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])

In [20]:
attn_scores_2_nm = torch.softmax(attn_scores_2, dim=0)


In [21]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_scores_2_nm[i]*x_i

print(context_vec_2)



tensor([0.4419, 0.6515, 0.5683])
